In [137]:
import numpy as np
import cv2

In [138]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2].astype(np.uint16) + depth_img[:, :, 1].astype(np.uint16) * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [139]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
	alpha, beta = K[0, 0], K[1, 1]
	cx, cy = K[0, 2], K[1, 2]
    
	# create a meshgrid for pixel coordinates
	h, w = depth.shape
	u, v = np.meshgrid(np.arange(w), np.arange(h))
	u, v = u.flatten(), v.flatten()
	z = depth.flatten()
	
	x = (u - cx) * z / alpha
	y = (v - cy) * z / beta
    
	# stack the coordinates and filter by segmentation
	pc = np.vstack((x, y, z)).reshape(depth.shape[0], depth.shape[1], 3)

	pc = pc[seg == 255]

	print('point cloud shape:', pc.shape)	

	return partial_pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

point cloud shape: (19375, 3)


In [140]:
# Save to .obj file and visualize in MeshLab, result can be found in ../results/partial_pc_visual_meshlab.png. 
def save_point_cloud_to_obj(pc, filename):
    """
    Save point cloud to an .obj file.
    Args:
        pc: numpy array of shape (N, 3), where N is the number of points.
        filename: str, the output .obj file path.
    """
    with open(filename, "w") as f:
        for point in pc:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")
    print(f"Point cloud saved to {filename}")


save_point_cloud_to_obj(partial_pc, "../results/partial_pc.obj")

Point cloud saved to ../results/partial_pc.obj


The MeshLab visulization of partial_pc is shown in the figure below. 

<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="../results/partial_pc_meshlab.png" alt="Partial Point Cloud" style="width: 45%; align: center;">
    </div>
</div>


In [141]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

print('partial pc shape:', partial_pc_sampled.shape)
print('full pc shape:', full_pc_sampled.shape)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------

# extent of partial point cloud and full point cloud
partial_pc_sampled = partial_pc_sampled[:, None, :]
full_pc_sampled = full_pc_sampled[None, :, :]

distance_norm =  np.linalg.norm(partial_pc_sampled - full_pc_sampled, axis=2)

# fix x in partial_pc_sampled, compute the min distance over full_pc_sampled
one_way_CD = np.sum(np.min(distance_norm, axis=1)) / partial_pc_sampled.shape[0]
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD[None])	# extand to 1D array to save

partial pc shape: (2048, 3)
full pc shape: (2048, 3)
one way chamfer distance:  0.009825480394133496


In [142]:
# The following code is for debugging, you can ignore it

partial = np.array([[1,1,1], [2,2,2]])
full = np.array([[3,4,5], [6,7,8], [9,10,11], [12,13,14]])

partial = partial[:, None, :]
full = full[None, :, :]

distance_norm =  np.linalg.norm(partial - full, axis=2)

print("distance_norm:", distance_norm)
print(np.min(distance_norm, axis=1))

distance_norm: [[ 5.38516481 10.48808848 15.65247584 20.83266666]
 [ 3.74165739  8.77496439 13.92838828 19.10497317]]
[5.38516481 3.74165739]


In [143]:
a = np.array([[1, 1, 1], [1, 1, 1]])
b = np.array([[3, 4, 5], [6, 7, 8]]) 

norm = np.linalg.norm(a - b, axis=1)
print('norm:', norm)

norm: [ 5.38516481 10.48808848]
